<span style="color:GREEN">This notebook contains the assignment given for week 3 in the data science capsotne.
1. getting the tabular ata from url given as df 
2. remove the cells with nan values - Borough column 
3. replace the  empty neighborhood value to borough name.
4. Printing shape of dataframe
5. adding co-ordinates to the locations given
<span>
    

<span style="color:blue"> Step 1: A. importing required libraries
    <span>

In [ ]:
#importing required libraries 
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import random 

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.extra.rate_limiter import RateLimiter

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

<span style="color:blue"> Step 1: 
    B.get all the content in the URL 
    C.get all the tables 
    D.then search for required table with the class name 
    E.load into dataframe
    <span>

In [ ]:
#the url used 
url_canada ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#assigning the page content 
page = urllib.request.urlopen(url_canada)
soup = BeautifulSoup(page, "lxml") # getting content in the format of beautifulsoup from HTML
print(soup.prettify()) # printing the content in string - HTML format 
soup.title

In [ ]:
all_tables =soup.find_all('table') # all tables data 
all_tables

In [ ]:
canada_table= soup.find('table',class_="wikitable")
canada_table

In [ ]:
#loading into dataframe
column_1= []
column_2=[]
column_3=[]

for row in canada_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        column_1.append(cells[0].find(text=True))
        column_2.append(cells[1].find(text=True))
        column_3.append(cells[2].find(text=True))
        
df=pd.DataFrame(column_1,columns=['Postal_Code'])
df['Borough']=column_2
df['Neighborhood']=column_3


In [ ]:
 df.head() #getting the data of top five rows

<span style="color:blue"> 
    STEP 2 : data preprocessing: <br />
    
    A. basic replacement -  1. removing"\n"(Borough) <br/>
                          2. "/" to comma(Neighborhood) <br />
    B. no value in Neighborhood to nan and not assigned in Borough to nan<span>

In [ ]:
df.replace(to_replace=['Not assigned','','\n','/'],
                  value=[np.nan,np.nan,"",","],inplace =True,regex=True) #replacing the values of the borough into nan and \n 

In [ ]:
#df[df.Borough != "Not assigned\n"] # one way to drop all rows with value given
df=df.dropna(subset = ["Borough"])#dropping na values 
print(df['Neighborhood'].isnull().any())
print(df['Borough'].isnull().any())
df.Neighborhood.fillna(df.Borough, inplace=True)#replacing values 

<span style="color:blue"> 
OUTPUT CROSSCHECK 
The values in the columns can bee seen her - no nan values in both borough and neighborhood columns 
<span>

In [ ]:
#getting unique value in the list 
borough_count =df['Borough'].unique().tolist() 
neighborhood_count=df['Neighborhood'].unique().tolist()

Printing the shape of data frame 

In [ ]:
df.shape

#df.to_csv('/Users/deepthi/DataScienceCapstone/canada_df.csv')

Data Analysis 


In [ ]:
# getting the number of borough and neighborhoods in respective borough
boroughs = len(borough_count)  #103
neighborhoods=len(neighborhood_count)
print(boroughs)

In [ ]:
#histogram of boroughs distribution
labels, counts = np.unique(df['Borough'],return_counts=True)
ticks=range(len(counts))
plt.bar(ticks,counts,align='center')
plt.xticks(ticks,labels,rotation=60)

I tried doing it via fore_square but I am getting wrong cordinates to few places and hence did merge the data frames.

In [ ]:
CLIENT_ID = '00IBPIQNSPGE4VKVDFLL0YYHAW51XQF0MXZO2JAK22GR3ZHQ' # your Foursquare ID
CLIENT_SECRET = 'BI5TAFTPWTQ5QGMFZQTJX00TTVRYR4DYEDYRGEFVBJHONZ0P' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
df['Address'] = df['Postal_Code']+',Toronto'


from opencage.geocoder import OpenCageGeocode
geocoder = OpenCageGeocode('770602f8439a467ca68a8d301c67db81')

list_lat =[]
list_long = []
for index, row in df.iterrows(): # iterate over rows in dataframe
    address= row['Address']
    results= geocoder.geocode(address)
    lat= results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    list_lat.append(lat)
    list_long.append(long)
    
df['Latitude'] = list_lat 
df['Longitude'] = list_long




In [200]:
#copying into tempeorory dat frame 
df_canada = df.copy()

In [212]:
df_cordinates = pd.read_csv("/Users/deepthi/DataScienceCapstone/geo.csv")
df_cordinates= df_cordinates.rename(columns={'Postal Code':'Postal_Code'})

In [213]:
df_cordinates.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [214]:
#merging two data frames 
df_merged =  pd.merge(df_canada, df_cordinates, on = "Postal_Code")

In [215]:
df_merged.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
